<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/06_Ajuste_Fino_Clasificacion_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ajuste Fino (Fine-Tuning) para Clasificación de Texto

 Este notebook muestra cómo adaptar un modelo de lenguaje pre-entrenado para realizar clasificación de sentimientos en reseñas de películas.


## 1) Instalación de Bibliotecas Necesarias

 Estas son las bibliotecas mínimas que necesitamos:
 - transformers: para trabajar con modelos de lenguaje
 - datasets: para cargar y manipular datos
 - torch: para el procesamiento con deep learning

In [1]:
!pip install transformers datasets torch

## 2) Importar Bibliotecas

In [2]:
import torch  # Biblioteca principal para deep learning
from transformers import (
    AutoModelForSequenceClassification,  # Para cargar el modelo pre-entrenado
    AutoTokenizer,  # Para procesar el texto
    Trainer,  # Para entrenar el modelo
    TrainingArguments,  # Para configurar el entrenamiento
)
from datasets import load_dataset  # Para cargar los datos
import numpy as np

## 3) Cargar Datos de Ejemplo

 Usamos un pequeño conjunto de reseñas de películas

In [3]:
print("Cargando datos de ejemplo...")
dataset = load_dataset("imdb", split="train[:200]")  # Solo 200 ejemplos para simplificar

# Dividimos los datos en entrenamiento y prueba
train_test = dataset.train_test_split(test_size=0.2)
print(f"Tamaño del conjunto de entrenamiento: {len(train_test['train'])}")
print(f"Tamaño del conjunto de prueba: {len(train_test['test'])}")


Cargando datos de ejemplo...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tamaño del conjunto de entrenamiento: 160
Tamaño del conjunto de prueba: 40


## 4) Preparar el Modelo Base

 Usamos un modelo BERT básico en inglés

In [4]:
print("\nCargando modelo base...")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2  # 2 clases: positivo y negativo
)


Cargando modelo base...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 5) Preparar los Datos

 Función para convertir texto a formato que entiende el modelo

In [16]:
def preparar_texto(ejemplos):
    return tokenizer(
        ejemplos["text"],
        truncation=True,  # Corta textos muy largos
        padding=True,     # Rellena textos cortos
        max_length=128    # Longitud máxima de cada texto
    )

print("\nPreparando datos...")
datos_procesados = train_test.map(preparar_texto, batched=True)


Preparando datos...


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

## 6) Configurar el Entrenamiento

 Configuración básica para el entrenamiento

In [6]:
argumentos_entrenamiento = TrainingArguments(
    output_dir="./resultados",      # Donde guardar resultados
    num_train_epochs=3,             # Número de pasadas por los datos
    per_device_train_batch_size=8,  # Ejemplos procesados a la vez
    per_device_eval_batch_size=8,
    logging_steps=10,               # Cada cuánto mostrar progreso
    report_to="none",              # Desactivar wandb y otros servicios de logging
)


## 7) Entrenar el Modelo

In [7]:
print("\nEntrenando el modelo...")
entrenador = Trainer(
    model=model,
    args=argumentos_entrenamiento,
    train_dataset=datos_procesados["train"],
    eval_dataset=datos_procesados["test"],
)

resultado_entrenamiento = entrenador.train()


Entrenando el modelo...


Step,Training Loss
10,0.187400
20,0.011800
30,0.002300
40,0.001000
50,0.000700
60,0.000600


## 8) Usar el Modelo

 Función simple para clasificar nuevo texto

In [8]:
# Función simple para clasificar nuevo texto
def clasificar_sentimiento(texto):
    # Detectar si hay GPU disponible
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)  # Mover el modelo al dispositivo correcto

    # Preparar el texto
    entradas = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
    # Mover las entradas al mismo dispositivo que el modelo
    entradas = {k: v.to(device) for k, v in entradas.items()}

    # Obtener predicción
    with torch.no_grad():
        salida = model(**entradas)
        prediccion = torch.nn.functional.softmax(salida.logits, dim=-1)

    # Mover el resultado a CPU para procesarlo
    prediccion = prediccion.cpu()

    # Interpretar resultado
    if prediccion[0][1] > 0.5:
        return "Positivo"
    else:
        return "Negativo"

# Ejemplo de uso
print("\nProbando el modelo:")
print(f"Dispositivo usado: {'GPU' if torch.cuda.is_available() else 'CPU'}")
ejemplos = [
    "This movie was fantastic! I really enjoyed it.",
    "What a terrible waste of time. I hated it."
]

try:
    for texto in ejemplos:
        sentimiento = clasificar_sentimiento(texto)
        print(f"\nTexto: {texto}")
        print(f"Sentimiento: {sentimiento}")
except Exception as e:
    print(f"\nOcurrió un error: {str(e)}")
    print("Por favor, asegúrate de que el modelo se haya entrenado correctamente.")


Probando el modelo:
Dispositivo usado: GPU

Texto: This movie was fantastic! I really enjoyed it.
Sentimiento: Negativo

Texto: What a terrible waste of time. I hated it.
Sentimiento: Negativo


# Mejoras

## 1) Carga Balanceada de Datos de Entrenamiento

Para obtener buenos resultados en tareas de clasificación, es muy importante tener un conjunto de datos balanceado (similar número de ejemplos para cada clase).
 Si no lo hacemos, el modelo podría aprender sesgos indeseados.

In [14]:
print("Cargando datos de ejemplo...")

# Cargamos ejemplos negativos y positivos por separado para asegurar el balance
negative_examples = load_dataset("imdb", split="train[:100]")  # 100 negativos
positive_examples = load_dataset("imdb", split="train[12500:12600]")  # 100 positivos

# Combinamos los ejemplos en un solo dataset
from datasets import concatenate_datasets
dataset = concatenate_datasets([negative_examples, positive_examples])

# Dividimos los datos en entrenamiento y prueba
train_test = dataset.train_test_split(test_size=0.2, seed=42)  # seed para reproducibilidad
print(f"\nTamaño del conjunto de entrenamiento: {len(train_test['train'])}")
print(f"Tamaño del conjunto de prueba: {len(train_test['test'])}")

Cargando datos de ejemplo...

Tamaño del conjunto de entrenamiento: 160
Tamaño del conjunto de prueba: 40


Verificamos la distribución de clases:

In [15]:
print("\nDistribución de clases en el dataset:")
import numpy as np
labels = dataset['label']
unique_labels, counts = np.unique(labels, return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"Clase {label} ({'Negativo' if label == 0 else 'Positivo'}): {count} ejemplos ({count/len(labels)*100:.1f}%)")



Distribución de clases en el dataset:
Clase 0 (Negativo): 100 ejemplos (50.0%)
Clase 1 (Positivo): 100 ejemplos (50.0%)


Ahora puedes continuar en el paso 5 - Preparar los Datos

## 2) Añadir métricas de evaluación durante el entrenamiento

Definimos las métricas de accuracy, f-measure (f1), precision y recall:

In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Configuramos el entrenamiento para que evalúe esas métricas cada 20 pasos:

In [12]:
argumentos_entrenamiento = TrainingArguments(
    output_dir="./resultados",          # Donde guardar resultados
    num_train_epochs=3,                 # Número de pasadas por los datos
    per_device_train_batch_size=8,      # Ejemplos procesados a la vez
    per_device_eval_batch_size=8,
    logging_steps=10,                   # Cada cuánto mostrar progreso
    eval_steps=20,                      # Cada cuántos pasos evaluar
    eval_strategy="steps",              # Evaluar durante el entrenamiento
    load_best_model_at_end=True,       # Cargar el mejor modelo al final
    metric_for_best_model="f1",        # Métrica para elegir el mejor modelo
    report_to="none",                  # Desactivar wandb
)

Entrenamos y evaluamos al mismo tiempo:

In [17]:
print("\nEntrenando el modelo...")
entrenador = Trainer(
    model=model,
    args=argumentos_entrenamiento,
    train_dataset=datos_procesados["train"],
    eval_dataset=datos_procesados["test"],
    compute_metrics=compute_metrics,
)

resultado_entrenamiento = entrenador.train()


Entrenando el modelo...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
20,0.041000,0.720396,0.875000,0.893617,0.875000,0.913043
40,0.001600,0.835547,0.875000,0.893617,0.875000,0.913043
60,0.000800,0.887140,0.875000,0.893617,0.875000,0.913043
